In [ ]:
%load_ext autoreload
%autoreload 2

#!pip install scikit-learn

# import the packages needed for plotting
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import os, json
from qPCR_plotting import *

In [ ]:
SELECTED_INDEX = 9

experiment_names = [f.split('.')[0] for f in os.listdir() if 'EEJR' in f]
print('All experiment files:', experiment_names)

# # run one experiment
# e = experiment_names[SELECTED_INDEX]
# print("Selected file", e, "at index", SELECTED_INDEX)
# df = analse_qpcr_experiment(experiment_name=e)

# df.to_json('qpcr_df_c2_01.json')
# for e in experiment_names:
#     try:
#         print('Analysing', e)
#         analse_qpcr_experiment(experiment_name=e)
#         print('Success')
#     except Exception as e:
#         print('Error', e)

In [ ]:

# run all experiments and make one dataframe
use_experiments = ['157', '158', '159', '160', '161', '162', '163', '164'] 
experiment_names = [e for e in experiment_names if any([u in e for u in use_experiments])]
print('Used experiment files:', experiment_names)

dfs = []

for e in experiment_names:
    output_folder = get_output_folder(experiment=e)
    df = load_dataframe(experiment_name=e)    
    standard_results = fit_to_standard(df, plot=False, output_folder=output_folder)
    plot_nort_controls(df, output_folder=output_folder)

    normalization_gene = 'Actin1'
    normalization_sample = 'NF54'
    normalization_sample = 'KOPfs25'

    df_cdna, sample_normalization_title, gene_normalization_title = add_concentration_and_normalizations(
        df=df.query('is_outlier == False'),
        standard_results=standard_results,
        normalization_gene=normalization_gene,
        normalization_sample=normalization_sample,
    )

    df_cdna_q = df_cdna.query('sample != @normalization_sample and gene != @normalization_gene')

    dfs.append(df_cdna_q)

df = pd.concat(dfs, ignore_index=True)
df.reset_index()

# df.to_json('qpcr_df_05_NF54.json')
# df.to_json('qpcr_df_05_KOPfs25.json')

df

In [ ]:
# run all experiments with KOP230P to normalize to
use_experiments = ['157', '158', '159', '160', '161', '162'] 
experiment_names = [e for e in experiment_names if any([u in e for u in use_experiments])]
print('Used experiment files:', experiment_names)

dfs = []

for e in experiment_names:
    output_folder = get_output_folder(experiment=e)
    df = load_dataframe(experiment_name=e)    
    standard_results = fit_to_standard(df, plot=False, output_folder=output_folder)
    plot_nort_controls(df, output_folder=output_folder)

    normalization_gene = 'Actin1'
    normalization_sample = 'KOP230P'

    df_cdna, sample_normalization_title, gene_normalization_title = add_concentration_and_normalizations(
        df=df.query('is_outlier == False'),
        standard_results=standard_results,
        normalization_gene=normalization_gene,
        normalization_sample=normalization_sample,
    )

    df_cdna_q = df_cdna.query('sample != @normalization_sample and gene != @normalization_gene')

    dfs.append(df_cdna_q)

df = pd.concat(dfs, ignore_index=True)
df.reset_index()

df.to_json('qpcr_df_05_KOP230P.json')

df

In [ ]:

# todo: add error bars

make_fold_change_plot(
    df=df,
    x_key='gene',
    y_key=sample_normalization_title,
    title=f"Gene Expression Fold Change to {normalization_sample} ({normalization_gene})",
    ylabel=f"Fold Change (normalized to {normalization_gene})",
    hue_key='sample',
    output_folder=output_folder,
    figsize=(20,10)
)